In [ ]:
!pip install numpy pandas scikit-learn tensorflow keras

In [ ]:
import os
import tarfile
import numpy as np
from sklearn.model_selection import train_test_split

#Load the Uploaded Files


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving rt-polarity.neg to rt-polarity.neg
Saving rt-polarity.pos to rt-polarity.pos


In [ ]:

# Function to load sentences from a file
def load_sentences(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as f:
        return [line.strip() for line in f]

# Load positive and negative sentences
positive_file_path = 'rt-polarity.pos'
negative_file_path = 'rt-polarity.neg'

positive_sentences = load_sentences(positive_file_path)
negative_sentences = load_sentences(negative_file_path)

print(f"Loaded {len(positive_sentences)} positive and {len(negative_sentences)} negative sentences.")



Loaded 5331 positive and 5331 negative sentences.


#Split the Data into Training, Validation, and Test Sets


In [ ]:
import numpy as np

# Labels: 1 for positive, 0 for negative
positive_labels = [1] * len(positive_sentences)
negative_labels = [0] * len(negative_sentences)

# Combine the data
sentences = positive_sentences + negative_sentences
labels = positive_labels + negative_labels

# Convert to numpy arrays for easy splitting
sentences = np.array(sentences)
labels = np.array(labels)

# Manual split without stratification (as per assignment instructions)
train_texts = np.concatenate([positive_sentences[:4000], negative_sentences[:4000]])
val_texts = np.concatenate([positive_sentences[4000:4500], negative_sentences[4000:4500]])
test_texts = np.concatenate([positive_sentences[4500:], negative_sentences[4500:]])

train_labels = np.concatenate([positive_labels[:4000], negative_labels[:4000]])
val_labels = np.concatenate([positive_labels[4000:4500], negative_labels[4000:4500]])
test_labels = np.concatenate([positive_labels[4500:], negative_labels[4500:]])

print(f"Training set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")
print(f"Test set size: {len(test_texts)}")


Training set size: 8000
Validation set size: 1000
Test set size: 1662


# Transform Text Data with TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

# Fit and transform the training data, transform validation and test data
X_train = vectorizer.fit_transform(train_texts).toarray()
X_val = vectorizer.transform(val_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

print("TF-IDF transformation complete.")
print("Training set shape:", X_train.shape)


TF-IDF transformation complete.
Training set shape: (8000, 5000)


# Build and Train the Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Define the neural network architecture
model = Sequential([
    Dense(128, input_shape=(5000,), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (positive/negative)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5555 - loss: 0.6804 - val_accuracy: 0.7450 - val_loss: 0.5308
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8187 - loss: 0.4192 - val_accuracy: 0.7640 - val_loss: 0.5038
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8904 - loss: 0.2817 - val_accuracy: 0.7550 - val_loss: 0.5561
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9297 - loss: 0.1946 - val_accuracy: 0.7540 - val_loss: 0.6456
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9613 - loss: 0.1250 - val_accuracy: 0.7540 - val_loss: 0.7557
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9769 - loss: 0.0791 - val_accuracy: 0.7450 - val_loss: 0.8689
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9864 - loss: 0.0441 - val_accuracy: 0.7390 - val_loss: 0.9795
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9938 - loss: 0.0277 - val_accura

#Evaluate the Model

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate on the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Precision, Recall, F1-score
report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'])
print("Classification Report:")
print(report)


52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion Matrix:
[[602 229]
 [237 594]]
Classification Report:
              precision    recall  f1-score   support

    Negative       0.72      0.72      0.72       831
    Positive       0.72      0.71      0.72       831

    accuracy                           0.72      1662
   macro avg       0.72      0.72      0.72      1662
weighted avg       0.72      0.72      0.72      1662

